In [1]:
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 10

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.set_num_threads(10)

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "CIFAR10",
            # 'dataset': "FMNIST",
            'random_class_idx': True,

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 100,
            # 'per_task_examples': np.inf,
            'per_task_examples': 10000,
            'per_task_memory_examples': 20,
            'batch_size_train': 256,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 10,
            # 'tau': 0.0,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.1,
            'learning_rate_decay_epoch': [30, 50, 70, 90],
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:7' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.001,
            'lambda': .01,
            'lambda_old': 0.0,
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/demo/dataset=CIFAR10/seed=10_epoch=100_lr=0.1_alpha=0.001_tau=10


In [2]:
from datasets import MNIST
from datasets import FashionMNIST
from datasets import CIFAR10, CIFAR100

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                    per_task_memory_examples=params['per_task_memory_examples'],
                    per_task_examples = params['per_task_examples'],
                    random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'FashionMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                            per_task_memory_examples=params['per_task_memory_examples'],
                            per_task_examples = params['per_task_examples'],
                            random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'CIFAR10':
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
elif params['dataset'] == 'CIFAR100':        
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
else:
    raise NotImplementedError
class_idx = benchmark.class_idx
num_classes = len(class_idx)



[8 2 5 6 3 1 0 7 4 9]
Files already downloaded and verified
Files already downloaded and verified


In [3]:
torch.get_num_threads()

10

In [4]:
num_classes

10

In [5]:
from algorithms.imbalance import Heuristic2
from metrics import MetricCollector2
from backbones import MLP2Layers2

from backbones import ResNet18Small2
backbone = ResNet18Small2(
    input_dim=input_dim, 
    output_dim=num_classes,
    class_idx=class_idx,
    config=params
    ).to(params['device'])
algorithm = Heuristic2(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

In [6]:
from trainers import ContinualTrainer

trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])


In [7]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 0.8955, 'loss': 0.0010577043145895005, 'std': 0.03450000000000003, 'EER': -1}
[2] Eval metrics for task 1 >> {'accuracy': 0.8965000000000001, 'loss': 0.001043150246143341, 'std': 0.0005000000000000004, 'EER': -1}
[3] Eval metrics for task 1 >> {'accuracy': 0.915, 'loss': 0.0009196268618106842, 'std': 0.0020000000000000018, 'EER': -1}
[4] Eval metrics for task 1 >> {'accuracy': 0.9099999999999999, 'loss': 0.000996010757982731, 'std': 0.03899999999999998, 'EER': -1}
[5] Eval metrics for task 1 >> {'accuracy': 0.9205000000000001, 'loss': 0.0008466195166110992, 'std': 0.003500000000000003, 'EER': -1}
[6] Eval metrics for task 1 >> {'accuracy': 0.888, 'loss': 0.0011598282009363174, 'std': 0.08199999999999996, 'EER': -1}
[7] Eval metrics for task 1 >> {'accuracy': 0.918, 'loss': 0.0008351377472281456, 'std': 0.0010000000000000009, 'EER': -1}
[8] Eval metrics for task 1 >> {'accuracy': 0.91

In [8]:
metric_manager_callback.meters['EER'].get_eer()

[0.0, 0.1865, 0.38422222222222224, 0.315125, 0.23364000000000001]

In [9]:
metric_manager_callback.meters['std'].get_std()

[0.009000000000000008,
 0.2092146027408221,
 0.4080785940096453,
 0.36503381692659653,
 0.30184141531605635]

In [10]:
metric_manager_callback.meters['accuracy'].get_data()

array([[0.977, 0.   , 0.   , 0.   , 0.   ],
       [0.259, 0.632, 0.   , 0.   , 0.   ],
       [0.003, 0.005, 0.868, 0.   , 0.   ],
       [0.004, 0.036, 0.01 , 0.857, 0.   ],
       [0.032, 0.203, 0.026, 0.173, 0.838]])

In [11]:
metric_manager_callback.meters['accuracy'].compute_overall()

0.4391633333333333

In [12]:
print(f"accuracy:{np.mean(metric_manager_callback.meters['accuracy'].compute_overall())}")
print(f"EER:{np.mean(metric_manager_callback.meters['EER'].compute_overall())}")
print(f"std:{np.mean(metric_manager_callback.meters['std'].compute_overall())}")


accuracy:0.4391633333333333
EER:0.22389744444444445
std:0.25863368579862406
